In [2]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('../../../'))
os.chdir(os.path.abspath('../../../'))

from dotenv import load_dotenv
import threading
from llm import LLMModels

from queue import Queue
import logging
from utils.llm import extract_answers
from utils.logging import CustomAdapter
import re 
import json
import cv2

from tests.experiment_notebooks.experiment_architecture import experiment_utils
import shutil
import os

### Select the parameters of the scene to recreate, choose a scene_name available

In [3]:
# Here you choose the exact simulation you want to simulate, note that .log file should be inside the logs folder

logs_folder = 'logs/'
simulation_file = "2024-02-06--12-31-06"
log_file = os.path.join(logs_folder, simulation_file, simulation_file + ".log")

# Here you choose the step range you want to simulate
step_init = 314
step_end = 370

# Here you are choosing the scene name and the path to the scene
scene_folder_name = "scene_agent_takes_last_two"
scene_path = "tests/experiment_notebooks/experiment_architecture/scenes/" + scene_folder_name + "/"

## Filter Log File

In [5]:
# Load the log file
with open(log_file, "r") as file:
    log = file.read()

filter_dict = experiment_utils.filter_log_by_agent(log, step_init, step_end)
log_by_agent, agents_scenes_description = filter_dict["log_by_agent"], filter_dict["agents_scenes_description"]

# Create the folder for the scene if it doesn't exist
if not os.path.exists(scene_path):
    os.makedirs(scene_path)

# Create a .log file for each agent, separate turns by "################Agent's" and add the header
for agent, logs in log_by_agent.items():
     
    with open(scene_path + agent + ".log", "w") as file:
        for log in logs:
            file.write("################Agent's " + agent + " turn##############" + log + "\n")

    
#Save scene description in a .json file for all agents: {agent: [scene_description]}
with open(scene_path + "agents_scenes_description.json", "w") as file:
    json.dump(agents_scenes_description, file)
    


300


### Recreate visually the scene

In [6]:

## Adapt the visual record
# Get the images from the simulation_folder + "world", then take images that are in the range of the steps, the format is "number.png"
# Create a video with the images and save it in the scene_folder


image_folder = logs_folder + simulation_file + "/world/"
video_path = scene_path + "scene.avi"


images = [img for img in os.listdir(image_folder) if img.endswith(".png") or img.endswith(".jpg")]

images.sort(key=lambda x: int(x.split('.')[0]))  # Asumiendo que el nombre del archivo es el "step" y no tiene puntos adicionales

out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'DIVX'), 1/1, (1600, 1200))

for image in images:
    if step_init <= int(image.split(".")[0]) <= step_end:
        img_path = image_folder + image
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (1600, 1200), interpolation=cv2.INTER_NEAREST)  # Redimensiona la imagen a la resolución deseada
        height, _, _ = img_resized.shape
        font = cv2.FONT_HERSHEY_SIMPLEX
        text = "Step: " + image.split(".")[0]
        cv2.putText(img_resized, text, (12, int(height * 0.035)), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
        out.write(img_resized)
        
    

out.release()